In [1]:
%matplotlib inline
import os

import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [2]:
os.getenv("DATASET_PATH")

'/Users/kayoonkim/Documents/winter-2023/data-literacy/city-recsys/all_data/all-data-best-city.csv'

In [3]:
final_df = pd.read_csv(os.getenv("DATASET_PATH"), index_col=0)

In [4]:
final_df['city_ascii'] = final_df['city_ascii'].str.strip()

In [10]:
import plotly.express as px
import geopy
from geopy.geocoders import Nominatim
import pandas as pd

# Initialize the geolocator
geolocator = Nominatim(user_agent="city_recommender")


# Get latitude and longitude for each city
city_coords = []
for city in final_df["city_ascii"]:
    location = geolocator.geocode(city)
    if location:
        city_coords.append({"Latitude": location.latitude, "Longitude": location.longitude})

# Add latitude and longitude data to the DataFrame
df_coords = pd.DataFrame(city_coords)
df = pd.concat([final_df, df_coords], axis=1)

fig_safety = px.scatter_geo(
    df, lat='Latitude', lon='Longitude', color='safety_index',
    color_continuous_scale='blues',  
    hover_name='city_ascii', labels={'safety_index': 'Safety Index'},
    title='Safety Index Visualization', projection='natural earth'
)
fig_safety.show()

fig_pollution = px.scatter_geo(
    df, lat='Latitude', lon='Longitude', color='pollution_index',
    color_continuous_scale='reds',  
    hover_name='city_ascii', labels={'pollution_index': 'Pollution Index'},
    title='Pollution Index Visualization', projection='natural earth'
)
fig_pollution.show()


In [11]:
final_df.columns

Index(['rank_cost_living', 'city_ascii', 'cost_live_rent_index',
       'purchase_pow_index', 'country', 'mean_tmp', 'std_dev_temp',
       'rank_safety', 'safety_index', 'pollution_index', 'rank_traffic',
       'trffic_min_index'],
      dtype='object')

In [12]:
%pip install dash

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/7b/62/438626ab498869249e99376684ba6d88931e7c2dd86d03a92d9d7a9b4c54/dash-2.14.2-py3-none-any.whl.metadata
  Obtaining dependency information for ansi2html from https://files.pythonhosted.org/packages/42/d7/1bc3433a2406b891182c25a7926abe3eb19af5870d540375ebf7795885b3/ansi2html-1.9.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 6.1 MB/s eta 0:00:0000:010:01m
Note: you may need to restart the kernel to use updated packages.


In [12]:
import plotly.express as px
from ipywidgets import interact, Dropdown
from geopy.geocoders import Nominatim

# Initialize the geolocator
geolocator = Nominatim(user_agent="city_recommender")


# Get latitude and longitude for each city
city_coords = []
for city in final_df["city_ascii"]:
    location = geolocator.geocode(city)
    if location:
        city_coords.append({"Latitude": location.latitude, "Longitude": location.longitude})

# Add latitude and longitude data to the DataFrame
df_coords = pd.DataFrame(city_coords)
df = pd.concat([final_df, df_coords], axis=1)

# Dropdown widget for selecting indices
dropdown = Dropdown(
    options=df.columns[1:], 
    description='Select Index:'
)

# Function to update the visualization based on selected index
def update_plot(selected_index):
    fig = px.scatter_geo(
        df, lat='Latitude', lon='Longitude', color=selected_index,
        color_continuous_scale='blues' if selected_index in ['safety_index', 'purchase_pow_index', 'trffic_min_index'] else 'reds',
        hover_name='City', labels={selected_index: selected_index.replace('_', ' ').title()},
        title=f'{selected_index.replace("_", " ").title()} Visualization', projection='natural earth'
    )
    fig.show()

# Interactive visualization based on dropdown selection
interact(update_plot, selected_index=dropdown)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Sofia&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))